<a href="https://colab.research.google.com/github/Harooniqbal4879/AgenticAI/blob/main/assignment_movie_review_sentiment_analysis_solution_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMDB movie review dataset sentiment analysis


The [IMDB movie review dataset](https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews) has 50k movie reviews. Each review has a binary sentiment classification label "positive" or "negative". We will use LLMs to perform movie review sentiment classification.

By the end of the assignment, you will gain hands on experience with
* Zero / Few shot the LLM to perform movie review sentiment analysis
* Evaluate accuracy of the sentiment predictions



In [ ]:
!pip install -q transformers==4.48.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 62.8 MB/s eta 0:00:00


In [ ]:
#!/bin/bash
! curl -L -o imdb-dataset-of-50k-movie-reviews.zip https://www.kaggle.com/api/v1/datasets/download/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
! unzip /content/imdb-dataset-of-50k-movie-reviews.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 25.7M  100 25.7M    0     0  39.7M      0 --:--:-- --:--:-- --:--:-- 39.7M
Archive:  /content/imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [ ]:
# This cell mounts the assignment data directory to allow file read.
# Please first add the class content directory "Module 6 : Deep Dive Into LLMs - V"
# as a short cut under your Google Drive, before you execute the cell. During
# execution, you will be prompted to give permissions to the drive mounting
# operation.

import os

# from google.colab import drive
# drive.mount('/content/drive')
# assets_dir = '/content/drive/MyDrive/Module 6 : Deep Dive into LLMs - V2/Assignment and MCQs/datasets/'

assets_dir = "/content/"

### Problem: Classify the sentiment of IMDB movie review as either `positive` or `negative`

In [ ]:
# Parse the csv data file "IMDB_Dataset.csv" into a data frame.
# The data frame has 2 columns, "review" and "sentiment":
# * review: The text of an IMDB movie review.
# * sentiment: Groundtruth sentiment label of the review. Has two possible
#     values: "positive" and "negative".
import pandas as pd

df_reviews = pd.read_csv(os.path.join(assets_dir, 'IMDB Dataset.csv'))

# view the first 3 rows of the data frame.
df_reviews.head(3)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive


In [ ]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [ ]:
# view full text of the first review in the data frame.
df_reviews.iloc[0].review

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [ ]:
import torch
import transformers

transformers.utils.logging.set_verbosity_error()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {device}")

Using cuda


In [ ]:
#@title Download phi-3 model and its tokenizer from hugging face

from transformers import AutoModelForCausalLM, AutoTokenizer

using_model = "microsoft/Phi-3-mini-4k-instruct"
# using_model = "microsoft/Phi-3.5-mini-instruct"

phi3_model = AutoModelForCausalLM.from_pretrained(using_model,
                                                  torch_dtype="auto",
                                                  trust_remote_code=True)
phi3_tokenizer = AutoTokenizer.from_pretrained(using_model)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py: 0.00B [00:00, ?B/s]

modeling_phi3.py: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

### Zero Shot and Few Shot Sentiment Analysis

* IMDB movie review sentiment analysis
We want to predict the sentiment of IMDB movie reviews as either "positive" or "negative". We can solve the problem with an LLM, by instructing the model using zero shot or few shot prompts, tailored for movie review sentiment analysis. Our goal is to make the model generate response strings equal to either "positive" or "negative".

* In zero-shot prompting, we instruct the model the generate text to complete the desired task. For example, for a translation task, we can write the following prompt:
```
Translate the sentence below to French.
Sentence: Today is sunny.
Translation:
```
Generally, we want the prompt to end in a instructional tone/wording that the model can understand as "it's now my turn to complete the task". In the translation prompt above, we do this by ending the prompt with "Translation:".

* In few-shot prompting, we also instruct the model the generate text to complete the desired task. But now we also additionally include a few examples. These examples should ideally be representative of the real data, and have sufficient coverage of desired model behavior. For example, suppose we want the model to judge whether an input is humurous, it helps to show the model a "yes" example and a "no" example. Figuring out the number of examples we include and the content of the examples are part of prompt engineering.
```
Judge whether the sentence is funny. Answer "yes" or "no". Here's some examples.
Sentence: Why was six afraid of seven? Because seven eight nine.
Answer:yes
Sentence: I had an omelette this morning.
Answer:no
Now it's your turn.
Sentence: Did you hear about the shepherd who drove his sheep through town? He was given a ticket for making a ewe turn.
Answer:
```

* Zero shot and few shot prompts for movie review sentiment analysis should include
  * Instructions describing the task to the model.
  * A few examples when using few shot.
  * The specific movie review we want to predict sentiment for.

### (YOUR CODE HERE) Complete the functions building zero shot and few shot prompts.
Python's string formatting (see https://realpython.com/python-string-formatting/ for details) is very helpful to combine instructions, examples along with specific reviews.
```
review = "5 stars"
formatted_prompt = "Rate this movie as {review}".format(review=review)
print(formatted_prompt)

>>> "Rate this movie as 5 stars"
```

In [ ]:
def zero_shot_prompt(review):
  return """Classify the sentiment of the follow movie review as either "positive" or "negative".

Review: {review}.
Sentiment:""".format(review=review)

def few_shot_prompt(review):
  return """You're given these two examples showing movie review sentiment.

Review: I had the pleasure of watching the Titanic movie. What a blast! Great story and cinematography.
Sentiment:positive

Review: The Room is a total waste of time. It's bad to the point of comical.
Sentiment:negative

Classify the sentiment of the follow movie review as either "positive" or "negative".
Review: {review}.
Sentiment:""".format(review=review)


### (YOUR CODE HERE) Complete `generation_args`

We have preprared a sentiment prediction function using Hugging face `transformers` package's `pipeline` object:. https://huggingface.co/microsoft/Phi-3-mini-4k-instruct shows example usage.

The `pipeline` object is initialized with the LLM model and its tokenizer. It takes a text prompt as input, and returns the model's text response. Under the hood, the `pipeline` tokenizes text input, calls the LLM, decode token ids from the LLM, and finally decodes token ids back to text. For the caller, this provides an easy "text in text out" interface with the LLM.

The `predict_sentiment` function below expects pre-loaded LLM model and tokenizer, as well as a zero/few shot prompt we built for movie review sentiment analysis. It returns the model's sentiment prediction for the  prompt in text.

Within the function, `generation_args` controls LLM decoding behavior. Complete the configuration of `generation_args` by replacing `None` with appropriate values if needed. You can find documentation of various generation config parameters in https://huggingface.co/docs/transformers/v4.42.0/en/main_classes/text_generation#transformers.GenerationConfig.

In [ ]:
from transformers import pipeline

def predict_sentiment(model, tokenizer, prompt):
  pipe = pipeline(
      "text-generation",
      model=model,
      tokenizer=tokenizer,
      device=device,
  )

  generation_args = {
      "max_new_tokens": 10,            # how many max tokens to decode from the LLM
      "return_full_text": False,       # only return response (and exclude input prompt)
      "do_sample": False,              # boolean variable. when False, perform greedy decode. otherwise decode temperature is specified by "temprature" config above.
  }

  messages = [{"role": "user", "content": prompt}]
  chat_prompt = phi3_tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
  return pipe(chat_prompt, **generation_args)[0]['generated_text']

In [ ]:
predict_sentiment(phi3_model, phi3_tokenizer, 'hello')

' Hello! How can I assist you today?'

### (YOUR CODE HERE) Compare Zero Shot vs Few Shot Sentiment Analysis  Accuracy

The IMDB moview review dataset has 50k examples. This is too much compute for the assignment. We will look at the first `n` examples in the dataset. Feel free to adjust `n` according to your compute budget.

Compte the two function below to run zero shot and few shot sentiment analysis, and compute accuracy for both methods.
* Which method is more accurate?
* Does accuracy change with prompt modification?

In [ ]:
def zero_shot_sentiment_analysis(n=100):
  n_correct = 0
  for _, row in df_reviews.iloc[0:n].iterrows():
    review = row.review
    sentiment = row.sentiment
    prediction = predict_sentiment(phi3_model, phi3_tokenizer,
                                   zero_shot_prompt(review))
    print(prediction)
    # true sentiment = negative
    # prediction = " NEGATIVE."
    # prediction.strip().lower(). = "negative."
    n_correct += prediction.strip().lower().startswith(sentiment)
  accuracy = n_correct / n
  print(f'zero shot {accuracy = }')
  return accuracy

In [ ]:
def few_shot_sentiment_analysis(n=100):
  n_correct = 0
  for _, row in df_reviews.iloc[0:n].iterrows():
    review = row.review
    sentiment = row.sentiment
    prediction = predict_sentiment(phi3_model, phi3_tokenizer,
                                   few_shot_prompt(review))
    print(prediction)
    n_correct += prediction.strip().lower().startswith(sentiment)
  accuracy = n_correct / n
  print(f'few shot {accuracy = }')
  return accuracy

In [ ]:
zero_shot_sentiment_analysis(n=10)

 positive
 positive
 positive
 negative
 positive
 positive
 positive
 negative
 negative
 positive
zero shot accuracy = 1.0


1.0

In [ ]:
few_shot_sentiment_analysis(n=10)

 negative
 positive
 positive
 negative
 positive
 positive
 positive
 Negative
 negative
 positive
few shot accuracy = 0.9


0.9

In [ ]:
df_reviews.review.iloc[0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [ ]:
def get_generation(model, tokenizer, prompt):
  pipe = pipeline(
      "text-generation",
      model=model,
      tokenizer=tokenizer,
      device=device,
  )

  generation_args = {
      "max_new_tokens": 1000,            # how many max tokens to decode from the LLM
      "return_full_text": True,       # only return response (and exclude input prompt)
      "do_sample": 3,              # boolean variable. when False, perform greedy decode. otherwise decode temperature is specified by "temprature" config above.
  }

  messages = [{"role": "user", "content": prompt}]
  chat_prompt = phi3_tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
  return pipe(chat_prompt, **generation_args)[0]['generated_text']

print(get_generation(phi3_model, phi3_tokenizer, f"Re-write this review, make it small: {df_reviews.review.iloc[0]}"))

<|user|>
Re-write this review, make it small: One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say

In [ ]:
def few_shot_prompt(review):
  return """You're given these two examples showing movie review sentiment.

Review:  Oz's brutal nature and explicit content captivate viewers from the start. This show isn't for everyone, particularly those sensitive to violence and graphic themes. Em City, the prison's experimental section, sees a mix of dangerous individuals, leading to constant conflicts and deceit. The main attraction to Oz lies in its fearless approach. Initially shocking, frequent viewers develop an appreciation for its stark realism and intricate storytelling. Repeated exposure makes the injustices and violence more palatable. Over time, one begins to recognize and adapt to the uncomfortable elements, experiencing a unique form of distraction through disturbing yet fascinating content.
Sentiment:positive

Review: The Room is a total waste of time. It's bad to the point of comical.
Sentiment:negative

Classify the sentiment of the follow movie review as either "positive" or "negative".
Review: {review}.
Sentiment:""".format(review=review)

few_shot_sentiment_analysis(n=10)

 positive
 positive
 positive
 negative
 positive
 positive
 positive
 Negative
 negative
 positive
few shot accuracy = 1.0


1.0

# LLM Architecture & Pre-training LLMs - MCQs

## Question 1

**What is a language model?**

* **A)** Transformer
* **B)** A Chatbot
* **C)** RNN
* **D)** A probabilistic model that computes the next token probability distribution for a given context.

**Solution:** D

**Explanation:** A language model computes the next token probability. Transformer and RNN are model architectures applicable to various data types and not limited to language inputs. A Chatbot is a possible application of language models.

---

## Question 2

You trained a uni-gram model on a tiny corpus: \[“the sun is red”, “the sea is blue”]. Using this model, what are the values of P(red | sea is) and P(green | sea is)?

* **A)** 0 and 0
* **B)** 0.5 and 0
* **C)** 0.125 and 0
* **D)** None of the above

**Solution:** C

**Explanation:** In uni-gram models, P(next\_token | context) = P(next\_token). In the corpus there are 8 total words; “red” occurred once and “green” was absent. Therefore P(red | sea is) = 1/8 = 0.125, and P(green | sea is) = 0.

---

## Question 3

You trained a bi-gram model on a tiny corpus: \[“the sun is red”, “the sea is blue”, “the sun is rising”, “the sky is blue”]. Using this model to generate text, which of the following is most likely to be generated?

* **A)** the sun
* **B)** the sun is blue
* **C)** the sun is rising
* **D)** the sun is red

**Solution:** B

**Explanation:** Calculating bi-gram probabilities from the corpus yields the highest overall probability for “the sun is blue” (≈0.335) compared to other sequences.

---

## Question 4

How can we train an N‑Gram model to summarise today’s news?

* **A)** We can train the N‑Gram model using open internet data.
* **B)** We can train the N‑Gram model using web pages in the news category.
* **C)** We can train the N‑Gram model using web pages in the news category, then use RAG: retrieve today’s breaking news, and use the N‑Gram model to generate text conditioned on the retrieved news.
* **D)** The N‑Gram model is inappropriate for this use case.

**Solution:** D

**Explanation:** N‑Gram models are purely count-based and cannot perform abstractive summarisation. They can only regurgitate seen n‑grams and lack semantic understanding.

---

## Question 5

Which of the following tasks is NOT appropriate for RNN?

* **A)** Weather forecast.
* **B)** Transcribe podcasts.
* **C)** Predict housing prices using its location, sqft and year built.
* **D)** Object tracking in videos.

**Solution:** C

**Explanation:** RNNs are designed for sequential data. Predicting housing prices from non-sequential numeric features does not require sequence modeling.

---

## Question 6

What of the following model architecture performs the best on long text inputs?

* **A)** Bag-of-words
* **B)** RNN
* **C)** Transformer
* **D)** N‑Gram

**Solution:** C

**Explanation:** Transformers use self-attention to capture long-range dependencies, making them superior on long texts.

---

## Question 7

WordVec embeddings are trained to capture the meaning of words. Suppose we have a word whose top 5 closest embeddings are “orange”, “blueberry”, “strawberry”, “watermelon”, and “banana”. Which word is it likely to be?

* **A)** Red
* **B)** Lemon
* **C)** Fish
* **D)** Church

**Solution:** B

**Explanation:** The nearest neighbors are all fruits, so the mystery word is most likely “lemon.”

---

## Question 8

What is the meaning of “transfer learning”?

* **A)** Pretrained model knowledge is transferred to the new model by training it on the same data set.
* **B)** Pretrained model knowledge is transferred to the new model by assigning the same weight to the new model as the pretrained model.
* **C)** Pretrained model knowledge is transferred to the new model by assigning a different weight to the new model as the pretrained model.
* **D)** Pretrained model knowledge is transferred to the new model by building a new model with the same architecture as the pretrained model.

**Solution:** B

**Explanation:** Transfer learning reuses pretrained weights directly, allowing the new model to leverage previously learned representations.

---

## Question 9

Which of the following is not a type of fine‑tuning process?

* **A)** Feature Based
* **B)** Supervised Fine Tuning
* **C)** Next Sentence Prediction
* **D)** Few Shot Learning

**Solution:** C

**Explanation:** Next Sentence Prediction is a pretraining objective, not a fine‑tuning method.

---

## Question 10

Masked Language Model objective used in LLMs tries to predict:

* **A)** Masked words by using unidirectional context
* **B)** Masked words by using bidirectional context in the decoder
* **C)** Masked words by using bidirectional context in both encoder and decoder
* **D)** Masked words by using bidirectional context in the encoder

**Solution:** D

**Explanation:** MLM hides tokens and predicts them using bidirectional encoder context.

---

## Question 11

Large Language Models are harder to train because:

* **A)** The self‑attention matrix becomes larger.
* **B)** Number of trainable parameters are larger due to more encoder and decoder blocks.
* **C)** Vocabulary size is larger for LLMs.
* **D)** Memory footprint of LLMs is bigger due to the use of high‑precision floats.

**Solution:** B

**Explanation:** The sheer number of parameters increases compute and memory requirements during training.

---

## Question 12

How could supervised fine tuning an LLM be impractical for a given situation?

* **A)** There is a lack of high‑quality labeled data for the downstream task.
* **B)** Lack of computation infrastructure to perform updates of the model weights.
* **C)** A traditional machine learning model performs better or just as well as the fine‑tuned LLM.
* **D)** All of the Above

**Solution:** D

**Explanation:** Data scarcity, compute cost, and simpler baselines can all make fine‑tuning impractical.

---

## Question 13

The LLM RoBERTa is trained using only the MLM objective and not with the NSP objective. What are the possible reasons?

* **A)** The NSP objective does not directly enhance text generation based on contextual understanding.
* **B)** NSP tasks may suffer from noisy sentence relationship signals.
* **C)** The binary classification in NSP could oversimplify learning.
* **D)** All of the above

**Solution:** D

**Explanation:** NSP can provide weak signals and oversimplify training; MLM alone better serves contextual understanding.

---

## Question 14

How can transfer learning help transformer models like BERT perform better in different situations, especially with new data types?

* **A)** Transfer learning doesn't help; models must be trained from scratch.
* **B)** Transfer learning allows models to use prior knowledge to quickly adapt to new tasks or domains.
* **C)** Transfer learning is only useful with large datasets.
* **D)** Transfer learning is theoretical and not practical.

**Solution:** B

**Explanation:** Reusing pretrained knowledge lets models adapt efficiently to new domains with less data.

---

## Question 15

We often hear that training an LLM from scratch is not feasible for most companies. Why?

* **A)** The cost of training an LLM can be very high.
* **B)** Not enough computational resources/infrastructure.
* **C)** Not enough relevant data.
* **D)** All of the above

**Solution:** D

**Explanation:** High compute costs, data requirements, and infrastructure needs make from‑scratch training prohibitive.

---

## Question 16

What is the role of tokenization in LLM training?

* **A)** A way for the LLM to fine‑tune pieces of text for downstream tasks.
* **B)** Not important—models interpret strings natively.
* **C)** Provides a structured way to break text into manageable pieces.
* **D)** All of the above

**Solution:** C

**Explanation:** Tokenization segments text into units the model can process efficiently.

---

## Question 17

What is one main disadvantage of RLHF?

* **A)** Can only be performed for LLMs with over 1 billion parameters.
* **B)** Requires many human‑labeled samples, which are expensive and tedious.
* **C)** It can only be used for encoder‑only transformers.
* **D)** None of the above

**Solution:** B

**Explanation:** RLHF depends on costly, time‑consuming human annotations.

---

## Question 18

LoRA used during training does the following:

* **A)** It is not a defense against catastrophic forgetting.
* **B)** Uses the intuition that models are over‑parameterized and weight changes lie in a low‑dimensional subspace.
* **C)** Leads to faster fine‑tuning due to fewer trainable parameters.
* **D)** B and C

**Solution:** D

**Explanation:** LoRA reduces trainable parameters by projecting weight updates into a low‑dimensional subspace, speeding fine‑tuning.

---

## Question 19

Which model architecture introduced the concept of Instruction Tuning?

* **A)** Low Rank Adaptation
* **B)** Parameter Efficient Fine Tuning
* **C)** Fine Tuned Language Net (FLAN)
* **D)** None of the above

**Solution:** C

**Explanation:** Google’s FLAN pioneered instruction tuning by fine‑tuning on task instructions, improving zero/few‑shot performance.

---

## Question 20

In Reinforcement Learning from Human Feedback (RLHF), what describes “reward hacking”?

* **A)** Exploits the reward function.
* **B)** Optimizes the loss function against the reward value.
* **C)** Adds bias into the model based on output data.
* **D)** All of the above

**Solution:** A

**Explanation:** Reward hacking occurs when the agent finds loopholes in the reward function to maximize reward without following intended behavior.
